lyft uber yellow year month day hour pulocationid weekday isholiday

In [1]:
import pandas as pd
# 讀取檔案資訊
datasite = '../TS6-201902_202306.parquet'
df = pd.read_parquet(datasite)

print(df.shape)
df.head()

(30273912, 13)


,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,TempTime,countN
0,lyft,2019,2,1,0,3,4,False,4,40.864294,-73.84651,2019-02-01 00:00:00,4.0
1,lyft,2019,2,1,1,3,4,False,2,40.864294,-73.84651,2019-02-01 01:00:00,2.0
2,lyft,2019,2,1,2,3,4,False,6,40.864294,-73.84651,2019-02-01 02:00:00,6.0
3,lyft,2019,2,1,3,3,4,False,1,40.864294,-73.84651,2019-02-01 03:00:00,1.0
4,lyft,2019,2,1,4,3,4,False,1,40.864294,-73.84651,2019-02-01 04:00:00,1.0


In [2]:
#清出基本格式
df_temp = df.copy()
df_temp['day'] =1
df_temp = df_temp.drop(['Name','count','lat','lon','countN','hour','TempTime','weekday','is_holiday'],axis=1)
df_temp['TempTime'] = pd.to_datetime(df_temp[['year', 'month','day']])
df_temp.drop_duplicates(inplace=True)
df_temp = df_temp.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
print(df_temp.shape)
df_temp

(13833, 5)


,year,month,day,PULocationID,TempTime
0,2019,2,1,1,2019-02-01
1,2019,3,1,1,2019-03-01
2,2019,4,1,1,2019-04-01
3,2019,5,1,1,2019-05-01
4,2019,6,1,1,2019-06-01
...,...,...,...,...,...
13828,2023,2,1,263,2023-02-01
13829,2023,3,1,263,2023-03-01
13830,2023,4,1,263,2023-04-01
13831,2023,5,1,263,2023-05-01


In [3]:
df['day'] =1
#lyft
df_lyft = df[df['Name'] == 'lyft']
df_lyft = df_lyft.groupby(['Name', 'year', 'month', 'day', 'PULocationID'])['count'].sum().reset_index()
df_lyft['TempTime'] = pd.to_datetime(df_lyft[['year', 'month', 'day']])
df_lyft = df_lyft.drop(['Name','year','month','day'],axis=1)
df_lyft = df_lyft.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_lyft = df_lyft.drop(['PULocationID','TempTime'],axis=1)
df_lyft.rename(columns={'count': 'lyft'}, inplace=True)
#uber
df_uber = df[df['Name'] == 'uber']
df_uber = df_uber.groupby(['Name', 'year', 'month', 'day', 'PULocationID'])['count'].sum().reset_index()
df_uber['TempTime'] = pd.to_datetime(df_uber[['year', 'month', 'day']])
df_uber = df_uber.drop(['Name','year','month','day'],axis=1)
df_uber = df_uber.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_uber = df_uber.drop(['PULocationID','TempTime'],axis=1)
df_uber.rename(columns={'count': 'uber'}, inplace=True)
#yellow
df_yellow = df[df['Name'] == 'yellow']
df_yellow = df_yellow.groupby(['Name', 'year', 'month', 'day', 'PULocationID'])['count'].sum().reset_index()
df_yellow['TempTime'] = pd.to_datetime(df_yellow[['year', 'month', 'day']])
df_yellow = df_yellow.drop(['Name','year','month','day'],axis=1)
df_yellow = df_yellow.sort_values(by=['PULocationID','TempTime'], ascending=[True, True]).reset_index(drop=True)
df_yellow = df_yellow.drop(['PULocationID','TempTime'],axis=1)
df_yellow.rename(columns={'count': 'yellow'}, inplace=True)

In [4]:
df_temp = pd.concat([df_lyft,df_uber,df_yellow, df_temp], axis=1)
df_temp = df_temp.drop(['TempTime'],axis=1)

In [5]:
df_temp.head()

,lyft,uber,yellow,year,month,day,PULocationID
0,8,20,159,2019,2,1,1
1,11,54,178,2019,3,1,1
2,7,11,233,2019,4,1,1
3,4,1,235,2019,5,1,1
4,8,5,239,2019,6,1,1


In [6]:
#放入地區
datasite_temp = '../taxi_zone_lat_lon/organized_taxi_zone_lat_lon.csv'
#創立一個 df_temp 讀經緯度資料
df_temp1 = pd.read_csv(datasite_temp)

df_temp1

,zone,LocationID,borough,lat,lon
0,Newark Airport,1,EWR,40.690243,-74.174270
1,Jamaica Bay,2,Queens,40.612163,-73.817643
2,Allerton/Pelham Gardens,3,Bronx,40.864294,-73.846510
3,Alphabet City,4,Manhattan,40.723853,-73.975209
4,Arden Heights,5,Staten Island,40.556678,-74.189803
...,...,...,...,...,...
258,Williamsburg (South Side),256,Brooklyn,40.710736,-73.962940
259,Woodlawn/Wakefield,259,Bronx,40.900107,-73.853635
260,Woodside,260,Queens,40.746439,-73.905907
261,World Trade Center,261,Manhattan,40.707456,-74.013983


In [7]:
df_temp = df_temp.merge(df_temp1[['LocationID', 'zone','borough']], left_on=['PULocationID'], right_on=['LocationID'], how='left')
df_temp = df_temp.drop(['LocationID'],axis = 1)
df_temp['datetime'] = pd.to_datetime(df_temp[['year', 'month', 'day']])
df_temp['total'] = df_temp['lyft'] + df_temp['uber'] + df_temp['yellow']
df_temp

,lyft,uber,yellow,year,month,day,PULocationID,zone,borough,datetime,total
0,8,20,159,2019,2,1,1,Newark Airport,EWR,2019-02-01,187
1,11,54,178,2019,3,1,1,Newark Airport,EWR,2019-03-01,243
2,7,11,233,2019,4,1,1,Newark Airport,EWR,2019-04-01,251
3,4,1,235,2019,5,1,1,Newark Airport,EWR,2019-05-01,240
4,8,5,239,2019,6,1,1,Newark Airport,EWR,2019-06-01,252
...,...,...,...,...,...,...,...,...,...,...,...
13828,27670,77331,59089,2023,2,1,263,Yorkville West,Manhattan,2023-02-01,164090
13829,34029,81148,66582,2023,3,1,263,Yorkville West,Manhattan,2023-03-01,181759
13830,29750,80298,63186,2023,4,1,263,Yorkville West,Manhattan,2023-04-01,173234
13831,31365,77012,64699,2023,5,1,263,Yorkville West,Manhattan,2023-05-01,173076


In [8]:
df = df_temp
import os
# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'.'
DataName = 'TS22-201902_202306'
# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/{DataName}.parquet')
    print('save!')

save!


---